## Partie 1 : POS Tagging Médical

In [ ]:
import spacy
import re

text = "La patiente âgée de 65 ans présente une toux persistante et une fièvre à 38.5°C. Le médecin prescrit de l'amoxicilline 500mg 3x/jour."

nlp = spacy.load("fr_core_news_sm")

# Regex pour matcher "3x/jour" (ou des variantes similaires)
pattern = re.compile(r"(\d+x/jour|\d+(?:\.\d+)?°C|\d+mg|\d+g)")

# Personnaliser le tokenizer avec token_match
def custom_tokenizer(nlp):
    return spacy.tokenizer.Tokenizer(
        nlp.vocab,
        token_match=pattern.match,
        rules=nlp.Defaults.tokenizer_exceptions,
        prefix_search=nlp.tokenizer.prefix_search,
        suffix_search=nlp.tokenizer.suffix_search,
        infix_finditer=nlp.tokenizer.infix_finditer
    )

nlp.tokenizer = custom_tokenizer(nlp)

doc = nlp(text)

for token in doc:
    print(f" Text : {token.text:<15} POS : {token.pos_:<15} Lemma : {token.lemma_:<15} Dependence : {token.dep_:<15}  Head: {token.head}")

 Text : La              POS : DET             Lemma : le              Dependence : det              Head: patiente
 Text : patiente        POS : NOUN            Lemma : patient         Dependence : nsubj            Head: présente
 Text : âgée            POS : VERB            Lemma : âger            Dependence : acl              Head: patiente
 Text : de              POS : ADP             Lemma : de              Dependence : case             Head: ans
 Text : 65              POS : NUM             Lemma : 65              Dependence : nummod           Head: ans
 Text : ans             POS : NOUN            Lemma : an              Dependence : obl:arg          Head: âgée
 Text : présente        POS : VERB            Lemma : présente        Dependence : ROOT             Head: présente
 Text : une             POS : DET             Lemma : un              Dependence : det              Head: persistante
 Text : toux            POS : ADP             Lemma : toux            Dependence : case    

In [6]:
symptomes = []
for token in doc:
    if token.pos_ =="NOUN":
        mods = [t.text for t in token.children if t.dep_ in ["amod", "acl"]]
        if mods:
            symptomes.append(f"{token.text} {' '.join(mods)}")

print(symptomes)

['patiente âgée']


In [7]:
traitements = []
for token in doc:
    if token.lemma_ == "prescrire":
        objets = [t for t in token.rights if t.pos_ == "NOUN"]
        for o in objets:
            meds = f"{token.text} {o.text}"
            quant = [t.text for t in o.children if t.pos_ == "NUM"]
            if quant:
                meds += " " + " ".join(quant)
            traitements.append(meds)
print(traitements)

['prescrit amoxicilline 500mg']


## Partie 2 : Extraction de Relations

### Sujet-Verbe-Objet

In [8]:
texte = "Le médecin arrête l'aspirine en raison de saignements."
from analyse_medicale import extraire_relations

extraire_relations(nlp,texte)

[('médecin', 'arrêter', 'aspirine'), ('saignements', 'raison', 'arrêter')]

### Gestion des Négations

In [9]:
from analyse_medicale import negation_extraction

negation_extraction(nlp,texte)

[('arrêter', False)]

## Partie 3 : Analyse de Dépendances

In [16]:
from spacy import displacy

plt = displacy.render(doc,style="dep",jupyter=True)
# Sauvegarde dans un fichier .svg
with open("graph_dependance.png", "w", encoding="utf-8") as f:
    f.write(plt)

TypeError: write() argument must be str, not None

In [10]:
from analyse_medicale import extraire_medicaments_posologie

extraire_medicaments_posologie(nlp,text)

[('amoxicilline', '500mg 3x/jour')]

## Test 1:

In [ ]:
phrases_test = [
"Le patient n' a pas refuse l'anticoagulant malgré son AVC récent.",
"Prescription : ibuprofène 400mg si douleur, maximum 3 comprimés/jour.",
"Pas d'antibiothérapie pour cette infection virale."
]

print("Extraction des relations :\n")
for t in phrases_test:
    print(extraire_relations(nlp,t))

Extraction des relations :

[('patient', 'refuser', 'anticoagulant'), ('patient', 'refuser', 'avc')]
[]
[]


In [ ]:
print("Extraction des negations :\n")
for t in phrases_test:
    print(negation_extraction(nlp,t))

Extraction des negations :

[('refuser', True)]
[]
[]


## Test 2 :

In [ ]:
phrases = [
    "Le médecin arrête l'aspirine en raison de saignements.",
    "Le patient prend du paracétamol chaque matin.",
    "Le docteur ne prescrit pas d'antibiotiques.",
    "La patiente reçoit une perfusion à cause de la déshydratation.",
    "On suspend le traitement pour éviter les effets secondaires."
]

print("Extraction des relations :\n")
for t in phrases:
    print(extraire_relations(nlp,t))

Extraction des relations :

[('médecin', 'arrêter', 'aspirine'), ('saignements', 'raison', 'arrêter')]
[('patient', 'prendre', 'paracétamol')]
[('docteur', 'prescrire', 'antibiotiques')]
[('patiente', 'recevoir', 'perfusion')]
[('on', 'suspendre', 'traitement')]


In [ ]:
print("Extraction des negations :\n")
for t in phrases:
    print(negation_extraction(nlp,t))

Extraction des negations :

[('arrêter', False)]
[('prendre', False)]
[('prescrire', True)]
[('recevoir', False)]
[('suspendre', False)]
